In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
#load funda data, correct for price outliers and display first 5 rows
funda = pd.read_csv("funda-amsterdam.csv", encoding="utf-8")
funda_corr = funda[funda["price"] > 100000]
funda_corr.head()

,Unnamed: 0,address,area,bedrooms,postal_code,posting_date,price,property_type,rooms,sale_date,url,year_built
0,0,Knollendamstraat 4 III/IV,108,2,1013 TN,4-6-2016,510000,apartment,3,23-6-2016,http://www.funda.nl/koop/verkocht/amsterdam/ap...,1906
1,1,Moerdijkstraat 47 1,47,1,1079 XM,22-6-2016,215000,apartment,2,5-7-2016,http://www.funda.nl/koop/verkocht/amsterdam/ap...,1938
2,2,Zeeburgerdijk 349,116,2,1095 AD,9-6-2016,325000,apartment,3,8-7-2016,http://www.funda.nl/koop/verkocht/amsterdam/ap...,2003
3,3,Brederodestraat 124 -1,58,2,1054 VH,14-5-2016,315000,apartment,3,2-6-2016,http://www.funda.nl/koop/verkocht/amsterdam/ap...,1910
4,4,Admiraal De Ruijterweg 409 III,63,1,1055 MD,14-6-2016,200000,apartment,3,23-6-2016,http://www.funda.nl/koop/verkocht/amsterdam/ap...,1906


In [5]:
funda_sub = funda_corr[['area',"bedrooms", "price", "property_type", "address"]]
funda_sub.head()

,area,bedrooms,price,property_type,address
0,108,2,510000,apartment,Knollendamstraat 4 III/IV
1,47,1,215000,apartment,Moerdijkstraat 47 1
2,116,2,325000,apartment,Zeeburgerdijk 349
3,58,2,315000,apartment,Brederodestraat 124 -1
4,63,1,200000,apartment,Admiraal De Ruijterweg 409 III


In [13]:
funda_sub[['apartment',' house']] =pd.get_dummies(funda_sub["property_type"])
funda_sub
#funda_sub["property_type"].head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,area,bedrooms,price,property_type,address,apartment,house
0,108,2,510000,apartment,Knollendamstraat 4 III/IV,1,0
1,47,1,215000,apartment,Moerdijkstraat 47 1,1,0
2,116,2,325000,apartment,Zeeburgerdijk 349,1,0
3,58,2,315000,apartment,Brederodestraat 124 -1,1,0
4,63,1,200000,apartment,Admiraal De Ruijterweg 409 III,1,0
5,125,2,515000,apartment,Bertrand Russellstraat 46,1,0
6,64,3,200000,apartment,Admiraal De Ruijterweg 417 I,1,0
7,155,3,599000,apartment,Deltastraat 10 -boven,1,0
8,85,2,249000,apartment,Ben van Meerendonkstraat 49,1,0
9,67,2,300000,apartment,Jacob van Lennepstraat 263 B,1,0


In [ ]:
y = funda_sub['price'] #we want to predeict price, it is our dependent variable
X = funda_sub[["area","bedrooms", "property_type"]]
#X = df.loc[:,'action':'metascore'] #this slices the dataframe to include all rows and the columns from "action" to "metascore"
X.index = funda_sub['address'] 
X.head()